# NB MANGLER AT FØJE TINGENE TIL CUDA

# Loading requirements

In [2]:
! pip install datasets

     |████████████████████████████████| 163kB 6.1MB/s 
     |████████████████████████████████| 17.7MB 232kB/s 
     |████████████████████████████████| 245kB 46.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
! pip install transformers

     |████████████████████████████████| 1.3MB 3.8MB/s 
     |████████████████████████████████| 1.1MB 21.9MB/s 
     |████████████████████████████████| 890kB 26.9MB/s 
     |████████████████████████████████| 2.9MB 32.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ba1939c4dc24a25d77a2009d6399d7196cbd3f69128d24860a459d6df10b50f7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
# Import things to get data to work
from datasets import load_from_disk
import sys
import os

# Loading data

In [5]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
sys.path.append(os.path.join('/content/drive/My Drive/deep_learning_project'))
data_path = 'drive/My Drive/deep_learning_project/data/'

In [14]:
data = load_from_disk(data_path)

In [15]:
print(data[0])

{'answer': 'york', 'paragraph': 'judi dench born in dorset and later moved to dublin where he was raised he met dench s mother while he was studying medicine at trinity college dublin dench attended the mount school a quaker independent secondary school in york and became a quaker her brothers one of whom was actor jeffery dench were born in tyldesley lancashire her niece emma dench is a roman historian and professor previously at birkbeck university of london and currently at harvard university career in britain dench has developed a reputation as one of the greatest actresses of the post war period primarily through her work in theatre which has been her forte throughout her career she has more than once been named number one in polls for britain s best actor early years', 'question': 'Where in England was Dame Judi Dench born?', 'question_id': 'tc_3'}


# Training Model

In [18]:
from math import ceil
import torch
from transformers import AutoTokenizer, BertModel, AdamW

In [24]:
def get_loss(sim):
    nll = -(torch.diagonal(sim) - torch.logsumexp(sim, dim = 1))
    return sum(nll)# return negative loss

In [21]:
# Defining parameters
n_sample = 9
batch_size = 3
n_batches = ceil(n_sample/batch_size)


lr = 5e-5
n_epochs = 3

In [19]:
# Subset data
train_sample = data.select(range(n_sample))

# Tokenize data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', padding = True)
train_sample = train_sample.map(lambda example: {
    'Q_input_ids': tokenizer(example['question'], padding = 'max_length')['input_ids'],
    'Q_attention_mask': tokenizer(example['question'], padding = 'max_length')['attention_mask'],
    'Q_token_type_ids': tokenizer(example['question'], padding = 'max_length')['token_type_ids'],
    'P_input_ids': tokenizer(example['paragraph'], padding = 'max_length')['input_ids'],
    'P_attention_mask': tokenizer(example['paragraph'], padding = 'max_length')['attention_mask'],
    'P_token_type_ids': tokenizer(example['paragraph'], padding = 'max_length')['token_type_ids']},
    batched = True, batch_size= batch_size)

#%% Change to pytorch format. 
train_sample.set_format(type = 'torch', 
                        columns = ['Q_input_ids', 'Q_attention_mask', 'Q_token_type_ids',
                                   'P_input_ids', 'P_attention_mask', 'P_token_type_ids'])

In [22]:
# Get pretrained model
model = BertModel.from_pretrained('bert-base-uncased')
optim = AdamW(model.parameters(), lr = lr)

In [25]:
#%% Yay, training here
epoch_loss = [None]*n_batches
train_loss = [None]*n_epochs

for epoch in range(n_epochs):
    print(f'epoch: {epoch}')
    trainloader = torch.utils.data.DataLoader(train_sample, batch_size=batch_size)
    trainloader = iter(trainloader)
    
    for i, batch in enumerate(trainloader):
        print(i)
        
        # (get extra observation in training set)
        
        # Make forward pass
        P_outputs = model(input_ids=batch['P_input_ids'], 
                          attention_mask=batch['P_attention_mask'], 
                          token_type_ids=batch['P_token_type_ids'])
        Q_outputs = model(input_ids=batch['Q_input_ids'], 
                          attention_mask=batch['Q_attention_mask'], 
                          token_type_ids=batch['Q_token_type_ids'])
        P_encoded_layers = P_outputs[0][:, 0, :]
        Q_encoded_layers = Q_outputs[0][:, 0, :]
    
    
        # Calculate similarity matrix
        sim_matrix = torch.matmul(Q_encoded_layers, P_encoded_layers.T)
        
        # Get loss
        loss = get_loss(sim_matrix)
        epoch_loss[i] = loss
        print(loss)
        
        # Update weights
        loss.backward()
        optim.step()
    
    train_loss[epoch] = sum(epoch_loss)
    print(f'loss: {train_loss[epoch]}')
    




epoch: 0
0
tensor(36.7055, grad_fn=<AddBackward0>)
1
tensor(8.7057, grad_fn=<AddBackward0>)
2
tensor(15.0357, grad_fn=<AddBackward0>)
loss: 60.44683837890625
epoch: 1
0
tensor(21.1634, grad_fn=<AddBackward0>)
1
tensor(19.0698, grad_fn=<AddBackward0>)
2
tensor(0.9998, grad_fn=<AddBackward0>)
loss: 41.233062744140625
epoch: 2
0
tensor(2.6203, grad_fn=<AddBackward0>)
1
tensor(2.1931, grad_fn=<AddBackward0>)
2
tensor(1.8779, grad_fn=<AddBackward0>)
loss: 6.6913604736328125


In [27]:
#%% Saving model
model_path = 'drive/My Drive/deep_learning_project/model/'
model.save_pretrained(model_path)